In [1]:
from __future__ import division
#
import os, sys
sys.path.append(os.getcwd()+'/..')
#
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#
from supports._setting import individual_detail_dir
# from supports.charts import histograms

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
Y09_general = pd.read_csv('%s/Y09-individual-general.csv' % (individual_detail_dir))
Y10_general = pd.read_csv('%s/Y10-individual-general.csv' % (individual_detail_dir))
#
Y09_prev_in = pd.read_csv('%s/Y09-individual-prev-in-ap.csv' % (individual_detail_dir))
Y10_prev_in = pd.read_csv('%s/Y10-individual-prev-in-ap.csv' % (individual_detail_dir))
Y09_prev_out = pd.read_csv('%s/Y09-individual-prev-out-ap.csv' % (individual_detail_dir))
Y10_prev_out = pd.read_csv('%s/Y10-individual-prev-out-ap.csv' % (individual_detail_dir))

In [3]:
# Filtering
HOUR, CENT = 60*60, 100
PROD_LIMIT = 65 / HOUR * CENT
#
Y09_general = Y09_general[(Y09_general['total-prod']  < PROD_LIMIT)]
Y10_general = Y10_general[(Y10_general['total-prod']  < PROD_LIMIT)]
#
Y09_prev_in = Y09_prev_in[(Y09_prev_in['ap-prod'] < PROD_LIMIT)]
Y10_prev_in = Y10_prev_in[(Y10_prev_in['ap-prod'] < PROD_LIMIT)]
#
Y09_prev_out = Y09_prev_out[(Y09_prev_out['ap-prod'] < PROD_LIMIT)]
Y10_prev_out = Y10_prev_out[(Y10_prev_out['ap-prod'] < PROD_LIMIT)]

In [4]:
# both years
Y09_did, Y10_did = set(Y09_general['did']), set(Y10_general['did'])
both_years_full_drivers = list(Y09_did.intersection(Y10_did))
#
Y09_general = Y09_general[Y09_general['did'].isin(both_years_full_drivers)]
Y10_general = Y10_general[Y10_general['did'].isin(both_years_full_drivers)]
#
Y09_prev_in = Y09_prev_in[Y09_prev_in['did'].isin(both_years_full_drivers)]
Y10_prev_in = Y10_prev_in[Y10_prev_in['did'].isin(both_years_full_drivers)]
#
Y09_prev_out = Y09_prev_out[Y09_prev_out['did'].isin(both_years_full_drivers)]
Y10_prev_out = Y10_prev_out[Y10_prev_out['did'].isin(both_years_full_drivers)]

In [5]:
Y09_general_gb, Y10_general_gb = Y09_general.groupby(['did']), Y10_general.groupby(['did'])
Y09_driver_general_prod = Y09_general_gb.mean()['total-prod'].to_frame('avg_total_prod').reset_index()
Y10_driver_general_prod = Y10_general_gb.mean()['total-prod'].to_frame('avg_total_prod').reset_index()
Y09_d_genprod_hour = {did : total_prod * HOUR / CENT for did, total_prod in Y09_driver_general_prod.values}
Y10_d_genprod_hour = {did : total_prod * HOUR / CENT for did, total_prod in Y10_driver_general_prod.values}
#
Y09_prev_in_gb, Y10_prev_in_gb = Y09_prev_in.groupby(['did']), Y10_prev_in.groupby(['did'])
Y09_prev_in_driver_ap_prod = Y09_prev_in_gb.mean()['ap-prod'].to_frame('avg_ap_prod').reset_index()
Y10_prev_in_driver_ap_prod = Y10_prev_in_gb.mean()['ap-prod'].to_frame('avg_ap_prod').reset_index()
Y09_pin_d_aprod_hour = {did : ap_prod * HOUR / CENT for did, ap_prod in Y09_prev_in_driver_ap_prod.values}
Y10_pin_d_aprod_hour = {did : ap_prod * HOUR / CENT for did, ap_prod in Y10_prev_in_driver_ap_prod.values}
#
Y09_prev_out_gb, Y10_prev_out_gb = Y09_prev_out.groupby(['did']), Y10_prev_out.groupby(['did'])
Y09_prev_out_driver_ap_prod = Y09_prev_out_gb.mean()['ap-prod'].to_frame('avg_ap_prod').reset_index()
Y10_prev_out_driver_ap_prod = Y10_prev_out_gb.mean()['ap-prod'].to_frame('avg_ap_prod').reset_index()
Y09_pout_d_aprod_hour = {did : ap_prod * HOUR / CENT for did, ap_prod in Y09_prev_out_driver_ap_prod.values}
Y10_pout_d_aprod_hour = {did : ap_prod * HOUR / CENT for did, ap_prod in Y10_prev_out_driver_ap_prod.values}

In [6]:
def ap_prod_diff_gb(gbs, Y09_ap_prods, Y10_ap_prods):
    Y09_pin_d_aprod_hour, Y09_pout_d_aprod_hour = Y09_ap_prods
    Y10_pin_d_aprod_hour, Y10_pout_d_aprod_hour = Y10_ap_prods
    ap_diffs, ap_diffs_v = [], []
    for gb in gbs:
        ap_impro, ap_decr = [], []
        ap_impro_v, ap_decr_v = [], []
        for did in gb:
            Y09_ap_prod, Y10_ap_prod = 0, 0
            if Y09_pin_d_aprod_hour.has_key(did) and Y09_pout_d_aprod_hour.has_key(did):
                Y09_ap_prod = (Y09_pin_d_aprod_hour[did] + Y09_pout_d_aprod_hour[did]) / 2
            elif Y09_pin_d_aprod_hour.has_key(did):
                Y09_ap_prod = Y09_pin_d_aprod_hour[did]
            elif Y09_pout_d_aprod_hour.has_key(did):
                Y09_ap_prod = Y09_pout_d_aprod_hour[did]
            #
            if Y10_pin_d_aprod_hour.has_key(did) and Y10_pout_d_aprod_hour.has_key(did):
                Y10_ap_prod = (Y10_pin_d_aprod_hour[did] + Y10_pout_d_aprod_hour[did]) / 2
            elif Y10_pin_d_aprod_hour.has_key(did):
                Y10_ap_prod = Y10_pin_d_aprod_hour[did]
            elif Y10_pout_d_aprod_hour.has_key(did):
                Y10_ap_prod = Y10_pout_d_aprod_hour[did]
            #
            if Y10_ap_prod - Y09_ap_prod >= 0:
                ap_impro.append(did)
                ap_impro_v.append(Y10_ap_prod - Y09_ap_prod)
            else:
                ap_decr.append(did)
                ap_decr_v.append(Y10_ap_prod - Y09_ap_prod)
        ap_diffs.append([ap_impro, ap_decr])
        ap_diffs_v.append([ap_impro_v, ap_decr_v])
    return ap_diffs, ap_diffs_v

In [7]:
prod_diff_order = []
for did, Y09_general_prod in Y09_d_genprod_hour.iteritems():
    Y10_general_prod = Y10_d_genprod_hour[did]
    prod_diff_order.append([Y10_general_prod - Y09_general_prod, did])
prod_diff_order.sort()
prod_diff_order.reverse()

In [8]:
improv_per, same_per, decr_per = (0.0, 0.05), (0.45, 0.55), (0.95, 1.0)
pers = [improv_per, same_per, decr_per]
improv_gb, same_gb, decr_gb = [], [], []
improv_gb_v, same_gb_v, decr_gb_v = [], [], []
gbs = [improv_gb, same_gb, decr_gb]
gbs_v = [improv_gb_v, same_gb_v, decr_gb_v]
num_drivers = len(prod_diff_order)
for i, (low_bound, upper_bound) in enumerate(pers):
    for diff, did in prod_diff_order[int(low_bound * num_drivers):int(upper_bound * num_drivers)]:
        gbs[i].append(did)
        gbs_v[i].append(diff)

In [9]:
ap_diffs, ap_diffs_v = ap_prod_diff_gb(gbs, (Y09_pin_d_aprod_hour, Y09_pout_d_aprod_hour), (Y10_pin_d_aprod_hour, Y10_pout_d_aprod_hour))

In [10]:
for i, (ap_impro, ap_decr) in enumerate(ap_diffs):
    total_num = len(ap_impro) + len(ap_decr)
    ap_impro_v, ap_decr_v = ap_diffs_v[i]
    print '%d (%.2f-%.2f) avg %.2f'% (len(gbs[i]), pers[i][0], pers[i][1], np.mean(gbs_v[i]))
    print '%d(%.2f) avg %.2f, %d(%.2f) avg %.2f'%(len(ap_impro), len(ap_impro) / total_num, np.mean(ap_impro_v), 
                                         len(ap_decr), len(ap_decr) / total_num, np.mean(ap_decr_v))
    print

219 (0.00-0.05) avg 8.46
163(0.74) avg 9.80, 56(0.26) avg -11.64

439 (0.45-0.55) avg 2.18
301(0.69) avg 7.64, 138(0.31) avg -10.59

220 (0.95-1.00) avg -4.15
137(0.62) avg 9.46, 83(0.38) avg -16.45



In [11]:
Y10_ap_prod_did = []
for did in both_years_full_drivers:
    if Y10_pin_d_aprod_hour.has_key(did) and Y10_pout_d_aprod_hour.has_key(did):
        Y10_ap_prod = (Y10_pin_d_aprod_hour[did] + Y10_pout_d_aprod_hour[did]) / 2
    elif Y10_pin_d_aprod_hour.has_key(did):
        Y10_ap_prod = Y10_pin_d_aprod_hour[did]
    elif Y10_pout_d_aprod_hour.has_key(did):
        Y10_ap_prod = Y10_pout_d_aprod_hour[did]
    Y10_ap_prod_did.append([Y10_ap_prod, did])
Y10_ap_prod_did.sort()
Y10_ap_prod_did.reverse()

In [12]:
improv_per, same_per, decr_per = (0.0, 0.05), (0.45, 0.55), (0.95, 1.0)
pers = [improv_per, same_per, decr_per]
improv_gb, same_gb, decr_gb = [], [], []
improv_gb_v, same_gb_v, decr_gb_v = [], [], []
gbs = [improv_gb, same_gb, decr_gb]
gbs_v = [improv_gb_v, same_gb_v, decr_gb_v]
num_drivers = len(Y10_ap_prod_did)
for i, (low_bound, upper_bound) in enumerate(pers):
    for diff, did in Y10_ap_prod_did[int(low_bound * num_drivers):int(upper_bound * num_drivers)]:
        gbs[i].append(did)
        gbs_v[i].append(diff)

In [13]:
ap_diffs, ap_diffs_v = ap_prod_diff_gb(gbs, (Y09_pin_d_aprod_hour, Y09_pout_d_aprod_hour), (Y10_pin_d_aprod_hour, Y10_pout_d_aprod_hour))

In [14]:
for i, (ap_impro, ap_decr) in enumerate(ap_diffs):
    total_num = len(ap_impro) + len(ap_decr)
    ap_impro_v, ap_decr_v = ap_diffs_v[i]
    print '%d (%.2f-%.2f) avg %.2f'% (len(gbs[i]), pers[i][0], pers[i][1], np.mean(gbs_v[i]))
    print '%d(%.2f) avg %.2f, %d(%.2f) avg %.2f'%(len(ap_impro), len(ap_impro) / total_num, np.mean(ap_impro_v), 
                                         len(ap_decr), len(ap_decr) / total_num, np.mean(ap_decr_v))
    print

219 (0.00-0.05) avg 49.84
190(0.87) avg 25.03, 29(0.13) avg -16.57

439 (0.45-0.55) avg 25.61
307(0.70) avg 4.11, 132(0.30) avg -11.79

220 (0.95-1.00) avg 17.36
82(0.37) avg 6.17, 138(0.63) avg -11.37



In [15]:
order = []
for did in both_years_full_drivers:
    order.append([Y10_d_genprod_hour[did], did])
order.sort()
order.reverse()

In [16]:
improv_per, same_per, decr_per = (0.0, 0.05), (0.45, 0.55), (0.95, 1.0)
pers = [improv_per, same_per, decr_per]
improv_gb, same_gb, decr_gb = [], [], []
improv_gb_v, same_gb_v, decr_gb_v = [], [], []
gbs = [improv_gb, same_gb, decr_gb]
gbs_v = [improv_gb_v, same_gb_v, decr_gb_v]
num_drivers = len(order)
for i, (low_bound, upper_bound) in enumerate(pers):
    for diff, did in order[int(low_bound * num_drivers):int(upper_bound * num_drivers)]:
        gbs[i].append(did)
        gbs_v[i].append(diff)

In [17]:
ap_diffs, ap_diffs_v = ap_prod_diff_gb(gbs, (Y09_pin_d_aprod_hour, Y09_pout_d_aprod_hour), (Y10_pin_d_aprod_hour, Y10_pout_d_aprod_hour))

In [18]:
for i, (ap_impro, ap_decr) in enumerate(ap_diffs):
    total_num = len(ap_impro) + len(ap_decr)
    ap_impro_v, ap_decr_v = ap_diffs_v[i]
    print '%d (%.2f-%.2f) avg %.2f'% (len(gbs[i]), pers[i][0], pers[i][1], np.mean(gbs_v[i]))
    print '%d(%.2f). avg. diff. %.2f, %d(%.2f) avg. diff. %.2f'%(len(ap_impro), len(ap_impro) / total_num, np.mean(ap_impro_v), 
                                         len(ap_decr), len(ap_decr) / total_num, np.mean(ap_decr_v))
    print

219 (0.00-0.05) avg 37.49
152(0.69). avg. diff. 10.92, 67(0.31) avg. diff. -14.27

439 (0.45-0.55) avg 25.80
296(0.67). avg. diff. 7.98, 143(0.33) avg. diff. -9.58

220 (0.95-1.00) avg 16.26
134(0.61). avg. diff. 11.15, 86(0.39) avg. diff. -16.30

